## Check if model converges at all

In [1]:
%load_ext autoreload

In [11]:
import os, sys, importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf

BASE_PATH = os.path.join(os.getcwd(), "..", "..")
MODULE_PATH = os.path.join(BASE_PATH, "modules")
DATASETS_PATH = os.path.join(BASE_PATH, "datasets")

sys.path.append(MODULE_PATH)

from active_learning import TrainConfig
from data import BenchmarkData, DataSetType
from models import setup_growth, default_model
from bayesian import BayesModel, McDropout, MomentPropagation

import mp.MomentPropagation as mp

In [3]:
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [4]:
benchmark = BenchmarkData(DataSetType.MNIST, os.path.join(DATASETS_PATH, "mnist"), classes=2)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(benchmark.inputs, benchmark.targets)

In [6]:
print("Datapoints: {}".format(len(benchmark.targets)))
print("Unique labels: {}".format(np.unique(benchmark.targets)))

Datapoints: 14780
Unique labels: [0 1]


In [7]:
model = default_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["binary_accuracy"])

In [8]:
model.evaluate(x_test, y_test)

116/116 [==============================] - 3s 22ms/step - loss: 3.3832 - binary_accuracy: 0.4612


[3.3832244873046875, 0.461163729429245]

# Model Debug

In [13]:
dp_model = BayesModel(model, TrainConfig())
#dp_model.fit(x_train, y_train, epochs=20, batch_size=60)

In [14]:
dp_model.get_model().evaluate(x_test, y_test)

116/116 [==============================] - 0s 2ms/step - loss: 3.3832 - binary_accuracy: 0.4612


[3.3832244873046875, 0.461163729429245]

In [15]:
%autoreload 2
mp_model = MomentPropagation(dp_model.get_model())
mp_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["binary_accuracy"])
mp_model.evaluate(x_test, y_test, batch_size=60)

[3.0234593484615537, 0.4638700947225981]

In [16]:
dp_model.fit(x_train[:10], y_train[:10], batch_size=10, epochs=5)
dp_model.evaluate(x_test, y_test)

Epoch 1/5
1/1 [==============================] - 0s 1ms/step - loss: 0.9502 - binary_accuracy: 0.9000
Epoch 2/5
1/1 [==============================] - 0s 1ms/step - loss: 4.6082 - binary_accuracy: 0.4000
Epoch 3/5
1/1 [==============================] - 0s 829us/step - loss: 7.2507 - binary_accuracy: 0.8000
Epoch 4/5
1/1 [==============================] - 0s 655us/step - loss: 2.3428 - binary_accuracy: 0.9000
Epoch 5/5
116/116 [==============================] - 0s 2ms/step - loss: 0.1644 - binary_accuracy: 0.9873


[0.16437587141990662, 0.9872801303863525]

In [17]:
mp_model = MomentPropagation(dp_model.get_model())
mp_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["binary_accuracy"])
mp_model.evaluate(x_test, y_test, batch_size=60)

[7.570039925779771, 0.4638700947225981]

In [18]:
dp_model.evaluate(x_test, y_test)

116/116 [==============================] - 0s 2ms/step - loss: 0.1644 - binary_accuracy: 0.9873


[0.16437587141990662, 0.9872801303863525]